In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#Importing libraries:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
import nltk.data
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
df = pd.read_excel('../input/blog-data/final_data_17406.xlsx')
df.head()


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
#df.drop(['id','date','img_src','section','topics','url'],axis=1,inplace=True)
df_main = df[['content','title','startups','mobile','apps','social','gadgets','europe','enterprise']]
df_main.dropna(inplace=True)


In [ ]:
df_main['content'] = df_main['content'].str.lstrip()
df_main['title'] = df_main['title'].str.lstrip()

In [ ]:
def checking_Data(index):
    row = df_main[df_main.index == index][['title', 'content']].values[0]
    if len(row) > 0:
        print('Title:',row[0])
        print('Content:',row[1])        
checking_Data(0)

In [ ]:
#Cleansing Content and Tags:
replace_blank_regex = re.compile('[/(){}\[\]\|@,;]')
symbols_removel_regex = re.compile('[^0-9a-z #+_$%-.,]')
stopword_det = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_blank_regex.sub(' ', text) # replace replace_blank_regex symbols by space in text
    text = symbols_removel_regex.sub('', text) # delete symbols which are in symbols_removel_regex from text
    #text = ' '.join(word for word in text.split() if word not in stopword_det) # delete stopwors from text
    return text
    
df_main['content'] = df_main['content'].apply(clean_text)
df_main['title'] = df_main['title'].apply(clean_text)
checking_Data(0)

In [ ]:
#Lemmatizing Content and Title:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lem_sentence=[]
    lem_sentence= [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

    return " ".join(lem_sentence) 
    
df_main['content_lemmatized'] = df_main.content.apply(lemmatize_text)
df_main['title_lemmatized'] = df_main.title.apply(lemmatize_text)

In [ ]:
#Combining Content+Title:
df_main['content+title'] = df_main['title_lemmatized']+' '+df_main['content_lemmatized']
df_main['content+title'].head()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, GlobalMaxPooling1D,SpatialDropout1D
from keras.layers import Embedding, GRU, Bidirectional
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)

DATA_PATH = '../input/'
EMBEDDING_DIR = '../input/'


MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

In [ ]:
#New Code for reading data:
#load data
train_df = df_main
print("num train: ", train_df.shape[0])


label_names = ['startups','mobile','apps','social','gadgets','europe','enterprise']
y_train = train_df[label_names].values

#visualize word distribution for Content+Title:
train_df['doc_len_content'] = train_df['content+title'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len_content'].mean() + train_df['doc_len_content'].std()).astype(int)
sns.distplot(train_df['doc_len_content'], hist=True,kde=True, 
             color='b', label='doc len content')
plt.axvline(x=max_seq_len, color='k', linestyle='--', label='max len')
plt.title('Content+Title length'); plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_df, y_train, test_size=0.2, random_state=42)

In [ ]:
#New Code for tokenizing "Content":
from keras.preprocessing.text import Tokenizer
tokenizer = RegexpTokenizer(r'\w+')
raw_docs_train = X_train['content+title'].tolist()
raw_docs_test = X_test['content+title'].tolist() 
 
num_classes = len(label_names)

print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))
#end for

print("pre-processing test data...")
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)#leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [ ]:
word_seq_train.shape

In [ ]:
#training params
batch_size = 256
num_epochs = 10

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4

In [ ]:
#embedding matrix for content:
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
embedding_matrix.shape

In [ ]:
print("sample words not found: ", np.random.choice(words_not_found, 10))

In [ ]:
#RNN architecture

print("training RNN-GRU ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(SpatialDropout1D(0.20))
model.add(Bidirectional(GRU(256, return_sequences=True, dropout=0.20, recurrent_dropout=0.1)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.20))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.20))
model.add(Dense(7, activation='sigmoid'))
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])

earlyStopping = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='min')
mcp_save = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, epsilon=1e-4, mode='min')

history = model.fit(word_seq_train, Y_train, validation_split=0.1, batch_size=128, 
                    epochs=10,callbacks=[earlyStopping,mcp_save,reduce_lr_loss])
model.summary()

from keras.models import load_model
model.save('Final_model.h5')


In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
from keras.models import load_model
Mod_Fin =load_model('../input/ece-657a/Final_model.h5')

In [ ]:
y_pred = Mod_Fin.predict(word_seq_test, batch_size=256, verbose=1)
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred.apply(lambda x: [0 if y <= 0.5 else 1 for y in x])
y_pred.head()

In [ ]:
###############################################################
#import os
#print(os.listdir("../input/gru-dropout-model"))

In [ ]:
#from keras.models import load_model
#model = load_model('../input/model-keras/my_model.h5')

In [ ]:
#Loss plots
plt.figure()
plt.plot(history.history['loss'], lw=2.0, color='b', label='train')
plt.plot(history.history['val_loss'], lw=2.0, color='r', label='val')
plt.title('GRU Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Categorical Accuracy plot:
plt.figure()
plt.plot(history.history['categorical_accuracy'], lw=2.0, color='b', label='train')
plt.plot(history.history['val_categorical_accuracy'], lw=2.0, color='r', label='val')
plt.title('GRU Training vs Validation Categorical Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Categorical Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
#Accuracy plot:
plt.figure()
plt.plot(history.history['acc'], lw=2.0, color='b', label='train')
plt.plot(history.history['val_acc'], lw=2.0, color='r', label='val')
plt.title('GRU Training vs Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
#Predicting:
y_pred = model.predict(word_seq_test, batch_size=256, verbose=1)
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred.apply(lambda x: [0 if y <= 0.5 else 1 for y in x])
y_pred.head()

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print("classification_report",classification_report(Y_test,y_pred))
print("accuracy_score",accuracy_score(Y_test,y_pred))
from sklearn.metrics import hamming_loss
print("hamming_loss",hamming_loss(Y_test,y_pred))


In [ ]:
#Confusion Matrix:
from scipy.sparse import csc_matrix
new_output = csc_matrix(y_pred,dtype=np.int8).toarray()
new_output

from sklearn.metrics import confusion_matrix
Acc = []
for i in range(1,8):
  cm_i = confusion_matrix(Y_test[:,i-1],new_output[:,i-1])
  Acc_i = (cm_i[0,0]+cm_i[1,1])/(cm_i[0,0]+cm_i[0,1]+cm_i[1,0]+cm_i[1,1])
  Acc.append(Acc_i)
  
print("Average Accuracy =", round((sum(Acc)/len(Acc)), 2))